In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="imgProductMain"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                if len(vehicle_url) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                oe = html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')
                oe = ';'.join([_.strip() for _ in oe])
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Type': type_,
                                         'Description': description,
                                         'Application_Summary': application_summary,
                                         'Vehicle': '',
                                         'OE': oe,
                                         'Url': url,
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])
                
                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./th/text()')[0].strip()[:-1].lower()] = tr.xpath('./td/text()')[0].strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：392

https://www.dormanproducts.com/p-132358-574-013.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [error] - 剩余数量：387
https://www.dormanproducts.com/p-132358-574-013.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [error] - 剩余数量：386
https://www.dormanproducts.com/p-132358-574-013.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [error] - 剩余数量：385


KeyboardInterrupt
2023-12-05T09:16:13Z


KeyboardInterrupt: 